In [10]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, metrics, regularizers
import numpy as np

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

tf.random.set_seed(1119001)

class ConvUnit(keras.Model):
    def __init__(self, channels, ksize, strides = 1, padding = "same"):
        super(ConvUnit, self).__init__()
        
        self.model = keras.models.Sequential([
            layers.Conv2D(channels, kernel_size = ksize, strides = strides, padding = padding, kernel_regularizer = regularizers.l2(1e-4), bias_regularizer = regularizers.l2(1e-4), activation = 'relu'),
            layers.BatchNormalization(),
        ])
        
    def call(self, x, training = None):
        x = self.model(x, training = training)
        
        return x
    
class MiniInception(keras.Model):
    def __init__(self, params):
        super(MiniInception, self).__init__()
        
        (ch1, ch2) = params
        
        self.conv1x1 = ConvUnit(ch1, ksize = (1, 1))
        self.conv3x3 = ConvUnit(ch2, ksize = (3, 3))
        
    def call(self, x, training = None):
        res1 = self.conv1x1(x, training = training)
        res2 = self.conv3x3(x, training = training)
        
        return tf.concat([res1, res2], axis = -1)
    
class DownSampling(keras.Model):
    def __init__(self, channels):
        super(DownSampling, self).__init__()
        
        self.conv = ConvUnit(channels, ksize = (3, 3), strides = 2, padding = "valid")
        self.pool = layers.MaxPooling2D(pool_size = (3, 3), strides = 2, padding = "valid")
        
    def call(self, x, training = None):
        res1 = self.conv(x, training = training)
        res2 = self.pool(x)
        
        return tf.concat([res1, res2], axis = -1)
    
class MiniGoogLeNet(keras.Model):
    def __init__(self, classes, **kwargs):
        super(MiniGoogLeNet, self).__init__()
        
        self.conv = ConvUnit(channels = 96, ksize = (3, 3))
        
        self.inception3a = MiniInception((32, 32))
        self.inception3b = MiniInception((32, 48))
        self.downsampling1 = DownSampling(80)
        
        self.inception4a = MiniInception((112, 48))
        self.inception4b = MiniInception((96, 64))
        self.inception4c = MiniInception((80, 80))
        self.inception4d = MiniInception((48, 96))
        self.downsampling2 = DownSampling(144)
        
        self.inception5a = MiniInception((176, 160))
        self.inception5b = MiniInception((176, 160))
        self.flatten = layers.GlobalAveragePooling2D()
        self.dropout1 = layers.Dropout(0.4)
        
        self.fc = layers.Dense(classes)
        self.dropout2 = layers.Dropout(0.4)
        self.softmax = layers.Dense(classes, activation = 'softmax')
                
    def call(self, x, training = None):
        x = self.conv(x, training = training)
        
        x = self.inception3a(x, training = training)
        x = self.inception3b(x, training = training)
        x = self.downsampling1(x, training = training)
        
        x = self.inception4a(x, training = training)
        x = self.inception4b(x, training = training)
        x = self.inception4c(x, training = training)
        x = self.inception4d(x, training = training)
        x = self.downsampling2(x, training = training)
        
        x = self.inception5a(x, training = training)
        x = self.inception5b(x, training = training)
        x = self.flatten(x)
        x = self.dropout1(x)
        
        x = self.fc(x, training = training)
        x = self.dropout2(x)
        x = self.softmax(x, training = training)
        
        return x
    
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

x_train = x_train.astype('float32')
mean_train = np.mean(x_train, axis = 0)
x_train = x_train - mean_train

x_test = x_test.astype('float32')
mean_test = np.mean(x_test, axis = 0)
x_test = x_test - mean_test

def one_hot(y, depth):
    res = np.zeros((len(y), depth))
    
    for i, label in enumerate(y):
        
        res[i, label] = 1
        
    return res

y_train = one_hot(y_train, depth = 10)
y_test = one_hot(y_test, depth = 10)

EPOCHS = 200
INIT_LR = 0.01

def poly_decay(epoch):
    maxEpochs = EPOCHS
    baseLR = INIT_LR
    power = 1.0
    
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
    
    return alpha

callbacks = [keras.callbacks.LearningRateScheduler(poly_decay)]

def main():
    model = MiniGoogLeNet(10)
    model.build(input_shape = (None, 32, 32, 3))
    model.summary()
    
    keras.utils.plot_model(model, to_file = 'MiniGooLeNet.png', show_shapes = True)
    
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(lr = INIT_LR, momentum = 0.9, nesterov = True), metrics = ['accuracy'])
    model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = EPOCHS, callbacks = callbacks)
        
if __name__ == '__main__':
    main()

Model: "mini_goog_le_net_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_unit_171 (ConvUnit)     multiple                  3072      
_________________________________________________________________
mini_inception_72 (MiniIncep multiple                  31040     
_________________________________________________________________
mini_inception_73 (MiniIncep multiple                  30096     
_________________________________________________________________
down_sampling_18 (DownSampli multiple                  58000     
_________________________________________________________________
mini_inception_74 (MiniIncep multiple                  87840     
_________________________________________________________________
mini_inception_75 (MiniIncep multiple                  108320    
_________________________________________________________________
mini_inception_76 (MiniIncep multiple           

50000/50000 [==============================] - 171s 3ms/sample - loss: 0.4362 - accuracy: 0.9395 - val_loss: 0.9402 - val_accuracy: 0.8587
Epoch 39/70
50000/50000 [==============================] - 171s 3ms/sample - loss: 0.4292 - accuracy: 0.9413 - val_loss: 0.9717 - val_accuracy: 0.8582
Epoch 40/70
50000/50000 [==============================] - 171s 3ms/sample - loss: 0.4226 - accuracy: 0.9445 - val_loss: 0.9795 - val_accuracy: 0.8551
Epoch 41/70
50000/50000 [==============================] - 171s 3ms/sample - loss: 0.4078 - accuracy: 0.9480 - val_loss: 1.0710 - val_accuracy: 0.8547
Epoch 42/70
50000/50000 [==============================] - 171s 3ms/sample - loss: 0.4040 - accuracy: 0.9470 - val_loss: 0.9427 - val_accuracy: 0.8667
Epoch 43/70
50000/50000 [==============================] - 171s 3ms/sample - loss: 0.3943 - accuracy: 0.9507 - val_loss: 0.9827 - val_accuracy: 0.8584
Epoch 44/70
50000/50000 [==============================] - 171s 3ms/sample - loss: 0.3918 - accuracy: 0.95